In [1]:
# Add testing url body text to csv to train a model:


In [2]:
import os, sys, re, time, gc, csv
read_loc = 'dataset/Raw/Read/'
store_loc = 'dataset/Raw/'
save_loc = f'{store_loc}LIWC/'

read_file = 'URL_Fake' # URL_True, True, Fake

Notebook_start = time.time()


is_URL = False
sufix = 'base'
folders = ['base_body','base_title']
if 'URL' in read_file:
    is_URL = True
    sufix = 'url'
    folders = ['url_body','url_title']
    

from LIWC_Methods import Custom_Methods 
CM = Custom_Methods()


import pandas as pd
import numpy as np


In [3]:
print(f'{read_loc}{read_file}.csv')
df_old = pd.read_csv(f'{read_loc}{read_file}.csv',error_bad_lines=False, quotechar='"', thousands=',' ,low_memory=False)

if is_URL:    
    idx = list(df_old[df_old.body != ' '].index)
    df_old = df_old.loc[idx]
else:
    idx = list(df_old[df_old.body != ' '].index)
    df_old = df_old.loc[idx]
    idx = list(df_old[df_old.title != ' '].index)
    df_old = df_old.loc[idx]
    
df_old.sort_values(by=['id'],inplace=True)
df_old.reset_index(drop=True,inplace=True)
df_old['title_url_min'] = ''
df_old['body_url_min'] = ''
df_old['title_url'] = ''
df_old['body_url'] = ''
df_old['title_stem'] = ''
df_old['body_stem'] = ''
df_old['link_cnt'] = 0
df_old['process_state'] = 'Pass'

df_old

dataset/Raw/Read/URL_Fake.csv


,id,news_url,source,title,body,class,title_url_min,body_url_min,title_url,body_url,title_stem,body_stem,link_cnt,process_state
0,Fake_kag_10,https://www.activistpost.com/2017/09/u-s-presi...,activistpost.com,U.S. President Donald Trump Quietly Signs Law ...,By Aaron Kesel\nAs former White House chief of...,1,,,,,,,0,Pass
1,Fake_kag_1003,http://beforeitsnews.com/sports/2017/09/puckin...,beforeitsnews.com,Puckin Hostile Shoutcast - Episode 90,Puckin Hostile Shoutcast – Episode 90\n% of re...,1,,,,,,,0,Pass
2,Fake_kag_1006,https://www.activistpost.com/2017/09/scientist...,activistpost.com,A Comprehensive Database on Chemtrails and Wea...,NaN,1,,,,,,,0,Pass
3,Fake_kag_1009,http://beforeitsnews.com/sports/2017/10/5-stag...,beforeitsnews.com,5 Stages of Grief - SAD!,5 Stages of Grief – SAD!\n% of readers think t...,1,,,,,,,0,Pass
4,Fake_kag_101,http://beforeitsnews.com/sports/2017/09/cowboy...,beforeitsnews.com,Cowboys WR Dez Bryant - Team Done with Anthem ...,Cowboys WR Dez Bryant – Team Done with Anthem ...,1,,,,,,,0,Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012,Fake_pol_95,http://patriotcrier.com/1-6kk-0/,patriotcrier.com,NaN,NaN,1,,,,,,,0,Pass
2013,Fake_pol_96,http://americannews.com/busted-nasa-accidental...,americannews.com,NaN,NaN,1,,,,,,,0,Pass
2014,Fake_pol_97,http://www.trendolizer.com/2017/10/actress-jen...,trendolizer.com,NaN,NaN,1,,,,,,,0,Pass
2015,Fake_pol_98,http://www.bluedotdaily.com/trump-police-offic...,bluedotdaily.com,NaN,NaN,1,,,,,,,0,Pass


In [4]:
len(df_old[df_old.id.str.contains('kag')])/len(df_old)

0.7873078829945463

In [5]:
len(df_old[df_old.id.str.contains('gos')])/len(df_old)

0.0

In [6]:
len(df_old[df_old.id.str.contains('pol')])/len(df_old)

0.21269211700545365

In [7]:
df_old.shape

(2017, 14)

In [8]:
i = 0
start = time.time()
last_start = start
save_text = True
    
end_cnt = len(df_old)
# end_cnt = 100


for idx, row in df_old[i:end_cnt].iterrows():
    if idx != -1:
        # print(idx,end=', ')
        gc.collect()
        if idx%250==0:
            last_start = CM.print_progress(idx,row,start,last_start)
        try:
            if is_URL:
                row = CM.scrap_page(row)
            else:
                row = CM.clean_text(row)
            if ("page not found" in row.title_url) or ('page unavailable' in row.title_url) or ("scrapper blocked by website" in row.title_url):
                row.process_state = 'Failed'
                save_text = False
                # print(f'failed to clean text for idx:{idx}, ID: {row["id"]}')

            df = row.copy().to_frame().T
            if pd.notnull(df.body_url[idx]):
                if ((len(df.body_url[idx]) > 25) and (save_text == True)): # for LIWC Processing
                    # print('Text Saved')
                    for part in ['title','body']:
                        col = ['id',f'{part}_url']
                        id_ = df.id[idx]
                        CM.save_LIWC_csv(save_loc,idx,df,part,sufix, id_)
                else:
                    # print(f'Text Not Saved for idx:{idx}, ID: {row["id"]}')
                    row.process_state = 'Failed'
                    save_text = True
            else:
                # print(f'df.body_url is null for idx:{idx}, ID: {row["id"]}')
                row.process_state = 'Failed'
                save_text = True
             
            
        except:
            pass
            # print(f'failed to scrap page for idx:{idx}, URL: {row["news_url"]}')
            
            
        CM.save_file(idx,df,read_file) 

end = time.time()
print(f'Total time in minutes: {int((end-start)/60)}', end=': ')

Total time:0  , Cycle time: 0.0, RowID:0 Attempting URL number:Fake_kag_10, URL: https://www.activistpost.com/2017/09/u-s-president-donald-trump-quietly-signs-law-allow-warrant-less-searches-parts-va-dc-md.html

Total time:10  , Cycle time: 9.6, RowID:250 Attempting URL number:Fake_kag_1579, URL: http://beforeitsnews.com/sports/2017/09/college-football-mel-kiper-and-todd-mcshays-list-of-most-exciting-players-to-watch-2701174.html

Total time:20  , Cycle time: 10.1, RowID:500 Attempting URL number:Fake_kag_213, URL: http://www.disclose.tv/news/warning_italian_supervolcano_on_the_brink_of_eruption/140334
Total time:31  , Cycle time: 10.8, RowID:750 Attempting URL number:Fake_kag_2725, URL: http://beforeitsnews.com/sports/2017/10/mitchell-trubiskys-debut-comes-at-the-right-time-of-the-schedule-2702044.html


Total time:41  , Cycle time: 10.2, RowID:1000 Attempting URL number:Fake_kag_3342, URL: http://beforeitsnews.com/sports/2017/09/jacoby-jones-to-retire-a-raven-2701446.html


Total tim

In [9]:
print('Done')

Done


In [10]:
loc = f'{store_loc}URL_Fake.csv'
print(loc)
df = pd.read_csv(loc)

# Replace the missing link_cnt values
avg_link_cnt = np.rint(np.mean(df.link_cnt))
print(avg_link_cnt)

avg_link_cnt = np.rint(np.mean(df.link_cnt[df.link_cnt>0]))
print(avg_link_cnt)
df.link_cnt = [np.rint(link_cnt) if link_cnt > 0 else avg_link_cnt for link_cnt in df.link_cnt]
df.process_state = ['Pass' if len(str_)> 50 else 'Failed' for str_ in df.body_url]
# save file
df.to_csv(loc,mode='w', header=True, index=False)
df = pd.read_csv(loc)
df.head()

dataset/Raw/URL_Fake.csv
57.0
65.0


,id,news_url,source,title,body,class,title_url_min,body_url_min,title_url,body_url,title_stem,body_stem,link_cnt,process_state
0,Fake_kag_10,https://www.activistpost.com/2017/09/u-s-presi...,activistpost.com,U.S. President Donald Trump Quietly Signs Law ...,By Aaron Kesel\nAs former White House chief of...,1,u.s. president donald trump quietly signs law ...,by aaron kesel as former white house chief of ...,u s president donald trump quietly signs law t...,by aaron kesel as former white house chief of ...,u presid donald trump quietli sign law allow w...,aaron kesel former white hous chief staff curr...,65.0,Pass
1,Fake_kag_1003,http://beforeitsnews.com/sports/2017/09/puckin...,beforeitsnews.com,Puckin Hostile Shoutcast - Episode 90,Puckin Hostile Shoutcast – Episode 90\n% of re...,1,scrapper blocked by website,scrapper blocked by website,scrapper blocked by website,scrapper blocked by website,scrapper block websit,scrapper block websit,27.0,Failed
2,Fake_kag_1006,https://www.activistpost.com/2017/09/scientist...,activistpost.com,A Comprehensive Database on Chemtrails and Wea...,NaN,1,scientist finally confirms weather modificatio...,by catherine j. frompovich too many humans in ...,scientist finally confirms weather modificatio...,by catherine j frompovich too many humans in t...,scientist final confirm weather modif comprehe...,catherin j frompovich mani human northern hemi...,65.0,Pass
3,Fake_kag_1009,http://beforeitsnews.com/sports/2017/10/5-stag...,beforeitsnews.com,5 Stages of Grief - SAD!,5 Stages of Grief – SAD!\n% of readers think t...,1,scrapper blocked by website,scrapper blocked by website,scrapper blocked by website,scrapper blocked by website,scrapper block websit,scrapper block websit,27.0,Failed
4,Fake_kag_101,http://beforeitsnews.com/sports/2017/09/cowboy...,beforeitsnews.com,Cowboys WR Dez Bryant - Team Done with Anthem ...,Cowboys WR Dez Bryant – Team Done with Anthem ...,1,scrapper blocked by website,scrapper blocked by website,scrapper blocked by website,scrapper blocked by website,scrapper block websit,scrapper block websit,27.0,Failed


In [11]:
len(df[df['process_state']=='Pass'])/len(df)

0.5731284085275161

In [12]:
len(df[df['process_state']=='Pass'])

1156

In [13]:
len(df)

2017

In [14]:
Notebook_end = time.time()
Total_Notebook_Time = round((Notebook_end - Notebook_start)/60.0,)
print(f'This notebook took {Total_Notebook_Time} min to run.')

This notebook took 84 min to run.
